In [24]:
import cobra
import pandas as pd
import os

base_path = os.getcwd()

model = cobra.io.load_json_model(os.path.join(base_path, "e_coli_core.json"))
expr_df = pd.read_csv(os.path.join(base_path, "e_coli_core_expression.csv"))
expr_df

,# Reaction ID,reaction activity [mmol/gDW/h]
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12
...,...,...
68,ME2,3.34
69,NADH16,20.26
70,NADTRHD,1.26
71,NH4t,3.45


In [25]:
import cobra
import pandas as pd
import os

base_path = os.getcwd()

model = cobra.io.load_json_model(os.path.join(base_path, "e_coli_core.json"))
expr_df = pd.read_csv(os.path.join(base_path, "e_coli_core_expression.csv"))
expr_df

,# Reaction ID,reaction activity [mmol/gDW/h]
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12
...,...,...
68,ME2,3.34
69,NADH16,20.26
70,NADTRHD,1.26
71,NH4t,3.45


In [26]:
expr_dict = dict(zip(expr_df["# Reaction ID"], expr_df[" reaction activity [mmol/gDW/h] "]))

# Constraints
for rxn in model.reactions:
    if rxn.id in expr_dict:
        value = expr_dict[rxn.id]

        if rxn.id == "EX_glc__D_e":  # glucose exchange → leave wide bounds
            continue

        if rxn.id == "ATPM":  # ATP maintenance → keep lower bound
            rxn.upper_bound = value
            continue

        if rxn.lower_bound < 0:  # reversible
            rxn.lower_bound = -value
            rxn.upper_bound = value
        else:  # irreversible
            rxn.upper_bound = value

# Collect bounds into a DataFrame
bounds_table = pd.DataFrame(
    [(rxn.id, rxn.lower_bound, rxn.upper_bound) for rxn in model.reactions],
    columns=["Reaction", "Lower bound", "Upper bound"]
)

print(bounds_table.to_string(index=False))

                Reaction  Lower bound  Upper bound
                     PFK         0.00        12.12
                     PFL         0.00         1.00
                     PGI       -13.12        13.12
                     PGK       -23.13        23.13
                     PGL         0.00         8.12
                   ACALD        -1.16         1.16
                  AKGt2r        -3.10         3.10
                     PGM       -20.01        20.01
                   PIt2r        -6.03         6.03
                  ALCD2x        -9.01         9.01
                  ACALDt        -2.29         2.29
                    ACKr        -1.19         1.19
                     PPC         0.00         2.56
                  ACONTa       -25.35        25.35
                  ACONTb       -25.35        25.35
                    ATPM         8.39      1000.00
                    PPCK         0.00        25.23
                   ACt2r        -3.23         3.23
                     PPS       

In [27]:
cobra.io.save_json_model(model, "e_coli_core_constrained.json") #saving the model for next exercises

### 4. a) Carry out an FBA optimization of biomass and report the maximal biomass production rate in the presence of the implemented constraints.

In [28]:
solution = model.optimize()
print(solution.objective_value)

0.48470309797054567


### b) Determine the “bottleneck(s)” – i. e. those reaction(s) whose flux reaches its maximum. 

In [29]:
#Bottleneck: flux = bound

for rxn in model.reactions:
    if abs(solution.fluxes[rxn.id] - rxn.upper_bound) < 1e-6 or abs(solution.fluxes[rxn.id] - rxn.lower_bound) < 1e-6:
        print(rxn.id, solution.fluxes[rxn.id], rxn.lower_bound, rxn.upper_bound)

PFL 1.0 0.0 1.0
ACALD -1.16 -1.16 1.16
ACKr -1.19 -1.19 1.19
ATPM 8.39 8.39 1000.0
PPCK 0.0 0.0 25.23
PPS 0.0 0.0 2.5
PYRt2 -1.26 -1.26 1.26
SUCCt2_2 0.0 0.0 2.36
SUCCt3 0.0 0.0 6.67
THD2 4.5 0.0 4.5
EX_acald_e 0.0 0.0 1000.0
EX_akg_e 0.0 0.0 1000.0
EX_fru_e 0.0 0.0 1000.0
EX_fum_e 0.0 0.0 1000.0
EX_gln__L_e 0.0 0.0 1000.0
EX_glu__L_e 0.0 0.0 1000.0
EX_mal__L_e 0.0 0.0 1000.0
EX_succ_e 0.0 0.0 1000.0
FBP 0.0 0.0 2.35
FORt2 0.0 0.0 1.28
FRD7 0.0 0.0 20.07
FRUpts2 0.0 0.0 10.08
FUMt2_2 0.0 0.0 2.45
GLNabc 0.0 0.0 0.25
GLUN 0.0 0.0 6.25
GLUSy 0.0 0.0 7.35
ICL 0.0 0.0 14.07
LDH_D -2.07 -2.07 2.07
MALS 0.0 0.0 9.12
MALt2_2 0.0 0.0 8.87
ME1 0.0 0.0 3.24
ME2 0.0 0.0 3.34
NADH16 20.26 0.0 20.26
NADTRHD 0.0 0.0 1.26


###

### The model does not use (have non-zero flux for) all reactions in the model constrained with maximal reaction activities. Pick a reaction with maximal reaction activity constraints and zero flux in the optimal solution, and explain from the network context why it carries not carry flux. (Hint: Use the online demo again.) 

FORt2 has an expression-based maximal activity but in the optimal solution its flux is 0. This makes sense because FBA only activates reactions if they are needed for biomass production. From the map, FORt2 would export formate, but under aerobic glucose conditions the cell doesn’t benefit from doing that. Just like in the tutorial where some knockouts didn’t affect growth because there were alternatives, here the model ignores FORt2 even though it has capacity.


### a) In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations. 

No, the maximal reaction activities are not equal in linear pathways. They vary a lot, because they reflect estimated enzyme capacity from gene expression, not steady-state flux balance. 

### a) Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions.

Exchange/transport reactions like EX_pyr_e, EX_for_e, EX_acald_e. These aren’t catalyzed by intracellular enzymes → no gene expression to map. Additionally, Non-enzymatic/pseudoreactions like FORti, these aren’t linked to specific genes, so expression data isn’t meaningful.